# Pre-processing

## Imports & Data Loading

In [89]:
from nltk.corpus import stopwords
import pandas as pd
import re

In [90]:
df = pd.read_pickle("models/data_original.pkl")

# undersample the data in 10%, distributing the classes proportionally in a stratified way and with a fixed seed
df = df.groupby("emotions", group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))

print(df.shape)

# print proportion of each class
print(df["emotions"].value_counts(normalize=True))


(41681, 2)
emotions
joy         0.338452
sadness     0.290756
anger       0.137521
fear        0.114465
love        0.082891
surprise    0.035916
Name: proportion, dtype: float64


## Cleaning

In [91]:
# Do some basic cleaning using regex

df['text'] = df['text'].str.lower()

# remove href link
df['text'] = df['text'].apply(lambda x: re.sub(r'href', '', x)) 
df['text'] = df['text'].apply(lambda x: re.sub(r'http', '', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'www', '', x))

# remove punctuation, numbers, and extra spaces
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'\n', ' ', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x))
df['text'] = df['text'].str.strip()


### Dealing with stopwords and negations

Because the dataset is very badly written, we need to consider words written with apostrophes, spaces or just connected to other words.

In [92]:
stop_words = set(stopwords.words('english'))

# if there is a stopword with an apostrophe, add the word without the apostrophe and a space instead of the apostrophe, and the word without the apostrophe
more_stop_words = set()
for word in stop_words:
    if "'" in word:
        more_stop_words.add(word.replace("'", " "))
        more_stop_words.add(word.replace("'", ""))
stop_words = stop_words.union(more_stop_words)

print(stop_words)

{'down', 'he', "you're", 'or', 'd', 'won', "doesn't", 'myself', 'has', 'each', 'more', 've', 'mightn', 'having', 'are', 'weren', 'won t', 'can', 'both', 'once', "it's", 'ours', 'few', 'into', 'wasn', 'than', 'yourselves', 'because', 'just', 'you ll', 'have', 'doesn', 'don', 'hadn t', 'there', "shan't", 'yourself', 'where', 'hasn', 'below', 'some', 'neednt', 'all', 'they', 'after', "mustn't", 'youve', 'hadnt', 'whom', 'doing', 'the', 'we', 'themselves', 'were', 'out', 'youre', 'it s', 'me', 'ma', 'over', 'while', 'its', 'y', 'mustn', 'shant', 'an', 'against', 'youd', 'very', 'between', 'a', 'their', 'him', 'any', 'if', 'under', 'himself', 'doesnt', 'had', 'own', 'hers', 'then', 'you d', 'most', "won't", 'aren t', 'for', 'shes', 'she', 'of', 'was', 'so', 'couldnt', 'too', 'at', 'no', 'o', 'i', 'needn', 'isnt', 'werent', 'about', 'been', 'weren t', "hadn't", 'havent', 'haven t', 'do', 'will', 'why', 'through', 'wont', 'to', 'hadn', 'shouldve', 'as', 'on', 'how', 'my', 'it', 'shan t', 'wha

In the case of negation words, we need to consider the words that are connected to them, so we can preserve the meaning of the sentence by preappending the negation word with a "not_" prefix. 

In [93]:
# remove negative words from stop_words list

neg_stop_words = {'no', 'nor', 'not', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", "can't", "cant"}

more_neg_words = set()
# for each negative word with an apostrophe, we will add the word without the apostrophe and a space instead of the apostrophe
for word in neg_stop_words:
    if "'" in word:
        more_neg_words.add(word.replace("'", " "))
        more_neg_words.add(word.replace("'", ""))

neg_stop_words = neg_stop_words.union(more_neg_words)

print(neg_stop_words)

# remove from stop_words the negative words
stop_words = stop_words - neg_stop_words


{'shouldn t', 'won', "doesn't", 'isn t', 'mustn t', 'mightn', 'weren', 'won t', 'mightnt', 'wasn', 'ain', 'mightn t', 'wouldnt', 'hadn t', 'doesn', 'can t', 'hasn t', "shan't", 'haven', 'hasn', "isn't", 'neednt', "wouldn't", "mustn't", 'hadnt', 'wouldn', "aren't", "needn't", 'ma', 'mustn', 'shant', 'not', 'arent', 'doesnt', 'couldn t', 'shouldnt', 'nor', "couldn't", "didn't", "shouldn't", 'doesn t', "won't", 'mustnt', 'cant', 'aren t', "wasn't", 'couldnt', 'no', "can't", 'needn', 'werent', 'isnt', 'hasnt', 'wasn t', 'weren t', 'didnt', "hadn't", "mightn't", 'wasnt', 'haven t', 'havent', 'wont', 'hadn', "hasn't", 'shan', "weren't", 'shouldn', 'wouldn t', 'aren', "haven't", 'didn t', 'shan t', 'needn t', 'didn', 'couldn', 'isn'}


In [94]:
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# if a word is in neg_stop_words, we will add a prefix "not_" to the word that follows it and remove the negative word
def add_not_prefix(text):
    words = text.split()
    new_words = []
    i = 0
    while i < len(words):
        if words[i] in neg_stop_words:
            try:
                new_words.append('not_' + words[i+1])
            except:
                new_words.append('not') # if the negative word is the last word, just add "not"
            i += 2
        else:
            new_words.append(words[i])
            i += 1
    return ' '.join(new_words)

df['text'] = df['text'].apply(add_not_prefix)

### Dealing with small words

Focusing now on small words, they are not very informative and are in many instances just noise. We can remove them from the dataset.

In [95]:
# Display percentage of words in the dataset that are less than 3 characters 

words = df['text'].str.split(expand=True).stack()

# Get set of unique small words
small_words = words[words.str.len() < 3]

print(f"The percentage of small words is {round(small_words[small_words.str.len() < 3].count() / words.count() * 100, 2)}%") 

# show the 30 most common small words
print(f"The 30 most common small words are:\n{small_words.value_counts().head(30)}") 

# The only interesting small words are the ones in keep_small_words
keep_small_words = {'go', 'ok', 'oh', 'pm', 'am', 'ex'}

# remove small words from the dataset and keep the ones in keep_small_words
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2 or word in keep_small_words]))

The percentage of small words is 2.65%
The 30 most common small words are:
im    6201
go    1027
us     433
id     373
ok     200
oh     119
u       72
tv      67
th      62
n       56
b       54
c       48
gt      47
e       45
w       42
pm      36
un      35
ex      33
lt      32
st      32
p       31
mr      27
en      25
x       25
dr      24
ya      24
co      22
f       21
ha      20
nd      20
Name: count, dtype: int64


### Dealing with bad written words

A thing that we noticed in the dataset is that there are a lot of words that are not written correctly. We can use a spell checker to correct them. But first, one thing that we can do is fix words that are written with more than one letter repeated, like "soooooo" or "amaaaaaazinggggg".

In [96]:
# remove all consecutive same letters
def replace_repetitive(text):
    pattern = r'(\w)\1{2,}'
    replaced_text = re.sub(pattern, r'\1\1', text)
    return replaced_text

df['text'] = df['text'].apply(replace_repetitive)

In [98]:
# Stemming and lemmatization

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def lemmatize(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def stem(text):
    return " ".join([stemmer.stem(word) for word in text.split()])


df["text"] = df["text"].apply(stem)
df["text"] = df["text"].apply(lemmatize)


In [99]:
# Show the first 10 rows
print(df["text"].head(10))

19132                           feel irrit kinda hate feel
51533    rather home feel violent lone not_tri sound in...
44351                   suggest wait discus feel le resent
51299                                wrong feel royal piss
55778    tierd talk like there hope hell care understan...
17018    feel frustrat honest like not_get money worth ...
45995    tri make chang feel urg happen particularli di...
45862                    truli good feel fight ever bother
48826                                          feel jealou
14602                       feel angri think like elsewher
Name: text, dtype: object


In [100]:
# save df to pickle
df.to_pickle("models/data_processed.pkl") # NOTE: ONLY SAVING 10% OF THE DATA